In [18]:
from traffic_generator import traffic_generator
import threading
import time

In [19]:
TOPOLOGY = ["Node_i", "Node_j", "Node_k", "Node_l", "Node_m"]
DATA_DICT = {
    "Node_i":[10, 20, 30, 40, 1],
    "Node_j":[15, 40, 25, 35, 1],
    "Node_k":[12, 18, 40, 25, 1],
    "Node_l":[40, 12, 15, 23, 1],
    "Node_m":[20, 13, 25, 11, 12]
}
link_bandwidth = 2 # 应该以邻接矩阵给出
traffic_distribution = "uniform"
link_delay = 5

In [ ]:
# Ring-AllReduce_algorithms

NODES_NUM = len(TOPOLOGY)
MAX_STEPS = 2 * (NODES_NUM - 1)
FINAL_DICT = DATA_DICT
for step in range(MAX_STEPS):
    REDUCE_FLAG = True
    if step >= (NODES_NUM - 1):
        REDUCE_FLAG = False
    # REDUCE_FLAG用于判断Ring-AllReduce处于Reduce阶段还是Broadcast阶段
    # Reduce阶段数据需要进行处理 (梯度聚合), Broadcast阶段数据只需要进行传输
    # 但是对于仿真而言传输数据量是固定的, 好像不需要考虑这个. 
    for node_id in range(NODES_NUM):
        data_index = ((node_id + step*(NODES_NUM-1)) % NODES_NUM)
        # print(data_index)
        if node_id == NODES_NUM-1:
            src_node = TOPOLOGY[node_id]
            dst_node = TOPOLOGY[0]
            if REDUCE_FLAG:
                FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
                continue
            FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]
            continue
        src_node = TOPOLOGY[node_id]
        dst_node = TOPOLOGY[node_id+1]
        if REDUCE_FLAG:
            FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
            continue
        FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]

print(FINAL_DICT)

{'Node_i': [97, 103, 135, 134, 16], 'Node_j': [97, 103, 135, 134, 16], 'Node_k': [97, 103, 135, 134, 16], 'Node_l': [97, 103, 135, 134, 16], 'Node_m': [97, 103, 135, 134, 16]}


In [12]:
# Broadcast_algorithms (Based on Binary-Tree)

father_nodes_list = ["Node_i"]
son_nodes_list = ["Node_j", "Node_k", "Node_l", "Node_m", "Node_n", "Node_o"]
data_size = 10000

link_bandwidth = 100
link_delay = 20
# 这里的link_delay和link_bd都应该从邻接矩阵里面取而不是直接给出
SUM_TIME = 0
NODES_CNT = 1 # 记录所有收到数据的节点个数, 这里也可以用log2来做因为是基于二叉树的, 但是本质没差. 加一个计数器好像更简单一些
while NODES_CNT < len(father_nodes_list) + len(son_nodes_list):
    temp_father_nodes_list = []
    max_time = 0
    # max_time用来记录每层Tree传输的时候的最大时间, 用这个最大时间作为该层的传输时间, 最终将多层的传输时间求和得到Broadcast的完成时间
    for father_node in father_nodes_list:
        for index, son_node in enumerate(son_nodes_list):
            time = data_size / link_bandwidth + link_delay
            # 正确的方法是这里应该根据不同的link_bd和link_delay计算出不同的两个node之间的时间
            if time > max_time:
                max_time = time
            NODES_CNT += 1
            _ = son_nodes_list.pop(index)
            temp_father_nodes_list.append(son_node)
            break
    SUM_TIME += max_time
    father_nodes_list += temp_father_nodes_list
print(SUM_TIME)

240.0


In [ ]:
from threading import Thread
import time

nodes_list = ["h1", "h2", "h3", "h4", "h5"]
node_status_matrix = {
    "h1": 1,
    "h2": 1,
    "h3": 1,
    "h4": 1,
    "h5": 1
}
# 0: waiting for data
# 1: ready to send
# 2: finished
def reduce_send(node_id, node_num, data_size):
    cnt = 0
    global nodes_list
    node_index = nodes_list.index(node_id)
    print(node_index)
    if node_index == len(nodes_list)-1:
        src_node = node_id
        dst_node = nodes_list[0]
    else:
        src_node = node_id
        dst_node = nodes_list[node_index+1]
    while True:
        if cnt == 2 * (node_num - 1):
            break
        global node_status_matrix
        if node_status_matrix[src_node] == 1:
            send_time = data_size / 2
            print(f"node_id:{node_id}, cnt:{cnt}")
            # ---
            # send
            time.sleep(node_index)
            node_status_matrix[src_node] = 0
            node_status_matrix[dst_node] = 1
            cnt += 1

# thread1 = Thread(target=reduce_send, args=("h1",5,10))
# thread2 = Thread(target=reduce_send, args=("h2",5,10))
# thread3 = Thread(target=reduce_send, args=("h3",5,10))
# thread4 = Thread(target=reduce_send, args=("h4",5,10))
# thread5 = Thread(target=reduce_send, args=("h5",5,10))


# thread1.start()
# thread2.start()
# thread3.start()
# thread4.start()
# thread5.start()

# thread1.join()
# thread2.join()
# thread3.join()
# thread4.join()
# thread5.join()


for i in range(len(nodes_list)):
    exec("thread{} = Thread(target=reduce_send, args=('h{}',5, 10))".format(i+1, i+1))

for i in range(len(nodes_list)):
    exec("thread{}.start()".format(i+1))

for i in range(len(nodes_list)):
    exec("thread{}.join()".format(i+1))


01
node_id:h2, cnt:0

node_id:h1, cnt:0
2
node_id:h3, cnt:0
3
node_id:h4, cnt:0
4
node_id:h5, cnt:0
node_id:h1, cnt:1
node_id:h2, cnt:1
node_id:h3, cnt:1
node_id:h4, cnt:1
node_id:h5, cnt:1
node_id:h1, cnt:2
node_id:h2, cnt:2
node_id:h3, cnt:2
node_id:h4, cnt:2
node_id:h5, cnt:2
node_id:h1, cnt:3
node_id:h2, cnt:3
node_id:h3, cnt:3
node_id:h4, cnt:3
node_id:h5, cnt:3
node_id:h1, cnt:4
node_id:h2, cnt:4
node_id:h3, cnt:4
node_id:h4, cnt:4
node_id:h5, cnt:4
node_id:h1, cnt:5
node_id:h2, cnt:5
node_id:h3, cnt:5
node_id:h4, cnt:5
node_id:h5, cnt:5
node_id:h1, cnt:6
node_id:h2, cnt:6
node_id:h3, cnt:6
node_id:h4, cnt:6
node_id:h5, cnt:6
node_id:h1, cnt:7
node_id:h2, cnt:7


KeyboardInterrupt: 

node_id:h3, cnt:7


node_id:h4, cnt:7
node_id:h5, cnt:7
